# MIPLIBインスタンスをダウンロードする

OMMXリポジトリでは、MIPLIB 2017の混合整数計画問題ベンチマークインスタンスをOMMX Artifact形式のデータとして提供しています。

```{note}
より詳細な説明：MIPLIB 2017のインスタンスに対応するOMMX ArtifactはOMMXリポジトリのGitHub コンテナーレジストリ ([link](https://github.com/Jij-Inc/ommx/pkgs/container/ommx%2Fmiplib2017))で管理されています。

GitHub コンテナーレジストリについては[こちら](https://docs.github.com/ja/packages/working-with-a-github-packages-registry/working-with-the-container-registry)を参照してください。
```

これらのインスタンスはOMMX SDKで簡単にダウンロードでき、OMMX Adapterの入力としてそのまま利用できます。
例えば、MIPLIB 2017のair05インスタンス ([参照](https://miplib.zib.de/instance_details_air05.html)) をPySCIPOptで解くには、以下の2ステップで実行できます：

1. OMMX Python SDKの`dataset`サブモジュールにある`miplib2017`関数で、air05インスタンスをダウンロードする。
2. ダウンロードしたインスタンスを、OMMX PySCIPOpt Adapterを介してPySCIPOptで解く。

具体的なPythonコードは以下の通りです：

In [1]:
# OMMX Python SDK
from ommx import dataset
# OMMX PySCIPOpt Adapter
from ommx_pyscipopt_adapter import OMMXPySCIPOptAdapter

# ステップ1: MIPLIB 2017のair05インスタンスをダウンロードする
instance = dataset.miplib2017("air05")

# ステップ2: OMMX PySCIPOpt Adapterを介してPySCIPOptで解く
solution = OMMXPySCIPOptAdapter.solve(instance)

この機能により、複数のOMMX対応ソルバーで同一のMIPLIBインスタンスを使用したベンチマーク測定を効率よく実行できます。

## 補足：インスタンスに付随するアノテーション

ダウンロードしたインスタンスには各種アノテーションが含まれており、`annotations` プロパティを使って全てのアノテーションにアクセスできます：

In [ ]:
import pandas as pd
# アノテーションを pandas を使って表形式で表示する
pd.DataFrame.from_dict(instance.annotations, orient="index", columns=["Value"]).sort_index()

インスタンスには、データセット共通のアノテーションとデータセット固有のアノテーションの2種類があります。

データセット共通のアノテーションには以下の7つがあり、それぞれに専用のプロパティが用意されています：

| アノテーション | プロパティ | 説明 |
|--------------|------------|------|
| `org.ommx.v1.instance.authors` | `authors` | そのインスタンスの作者 |
| `org.ommx.v1.instance.constraints` | `num_constraints` | そのインスタンスにある制約条件の数 |
| `org.ommx.v1.instance.created` | `created` | そのインスタンスがOMMX Artifact形式で保存された日時 |
| `org.ommx.v1.instance.dataset` | `dataset` | そのインスタンスが属するデータセット名 |
| `org.ommx.v1.instance.license` | `license` | そのデータセットのライセンス |
| `org.ommx.v1.instance.title` | `title` | そのインスタンスの名前 |
| `org.ommx.v1.instance.variables` | `num_variables` | そのインスタンスにある決定変数の総数 |

MIPLIBに特有のアノテーションは `org.ommx.miplib.*` というプレフィックスを持ちます。

例として、air05インスタンスの最適値を確認してみましょう。そのインスタンスの詳細ページ ([link](https://miplib.zib.de/instance_details_air05.html)) によれば、最適値は `26374` であり、この値はキー `org.ommx.miplib.objective` で取得できます：

In [ ]:
# アノテーションの値はすべて文字列 (str) であることに注意する！
assert instance.annotations["org.ommx.miplib.objective"] == "26374"

これにより、先ほどのOMMX PySCIPOpt Adapterで得られた計算結果が、期待される最適値と一致することを検証できます：

In [ ]:
import numpy as np

best = float(instance.annotations["org.ommx.miplib.objective"])
assert np.isclose(solution.objective, best)

# QPLIBインスタンスをダウンロードする

MIPLIB 2017に加えて、OMMXリポジトリではQPLIBの二次計画問題ベンチマークインスタンスもOMMX Artifact形式で提供しています。

```{note}
より詳細な説明：QPLIBインスタンスに対応するOMMX ArtifactはOMMXリポジトリのGitHub コンテナーレジストリ ([link](https://github.com/Jij-Inc/ommx/pkgs/container/ommx%2Fqplib))で管理されています。

QPLIBは二次計画問題のインスタンスライブラリです。QPLIBの詳細については [QPLIB website](http://qplib.zib.de/) を参照してください。
```

```{note}
**後方互換性に関する注記**: このノートブックのファイル名には歴史的な理由とURL互換性のため「miplib」が含まれていますが、現在はMIPLIBとQPLIBの両方のインスタンスを扱っています。
```

MIPLIBインスタンスと同様に、QPLIBインスタンスもOMMX SDKで簡単にダウンロードでき、OMMX Adapterで利用できます。
例えば、QPLIB_0018インスタンス ([参照](http://qplib.zib.de/QPLIB_0018.html)) をPySCIPOptで解くには、以下の2ステップで実行できます：

1. OMMX Python SDKの`dataset.qplib`関数で、0018インスタンスをダウンロードする。
2. ダウンロードしたインスタンスを、OMMX PySCIPOpt Adapterを介してPySCIPOptで解く。

具体的なPythonコードは以下の通りです：

In [ ]:
# OMMX Python SDK
from ommx import dataset
# OMMX PySCIPOpt Adapter
from ommx_pyscipopt_adapter import OMMXPySCIPOptAdapter

# ステップ1: QPLIBの0018インスタンスをダウンロードする
qp_instance = dataset.qplib("0018")

# ステップ2: OMMX PySCIPOpt Adapterを介してPySCIPOptで解く
qp_solution = OMMXPySCIPOptAdapter.solve(qp_instance)

この機能により、同一のQPLIBインスタンスを使用した二次計画問題ソルバーのベンチマーク測定を効率よく実行できます。

## QPLIBインスタンスのアノテーション

MIPLIBインスタンスと同様に、QPLIBインスタンスにも `annotations` プロパティを使ってアクセスできるアノテーションが含まれています：

In [ ]:
import pandas as pd
# QPLIBアノテーションを pandas を使って表形式で表示する
pd.DataFrame.from_dict(qp_instance.annotations, orient="index", columns=["Value"]).sort_index()

QPLIBインスタンスには、MIPLIBインスタンスと同じ7つのデータセット共通アノテーションに加えて、`org.ommx.qplib.*` プレフィックスを持つQPLIB固有のアノテーションがあります。

## MIPLIBとQPLIBアノテーションの比較

以下の表は、MIPLIBとQPLIBインスタンスの主要なアノテーションの違いを比較したものです：

| 項目 | MIPLIBアノテーション | QPLIBアノテーション |
|------|-------------------|------------------|
| 変数の種類 | `org.ommx.miplib.binaries`, `org.ommx.miplib.integers`, `org.ommx.miplib.continuous` | `org.ommx.qplib.nbinvars`, `org.ommx.qplib.nintvars`, `org.ommx.qplib.ncontvars` |
| 問題の種類 | `org.ommx.miplib.tags` (問題の特性を含む) | `org.ommx.qplib.probtype` (QCL, QML, QBL, LMQ等) |
| 目的関数 | `org.ommx.miplib.objective` (最適値) | `org.ommx.qplib.objtype`, `org.ommx.qplib.objcurvature` |
| 制約条件 | `org.ommx.miplib.non_zero` (制約行列の密度) | `org.ommx.qplib.nlincons`, `org.ommx.qplib.nquadcons` |
| 曲率 | 該当なし | `org.ommx.qplib.conscurvature` (制約の曲率) |

例として、QPLIBインスタンスの問題の種類と目的関数の曲率を確認できます：

In [ ]:
# QPLIB固有のアノテーション
print(f"問題の種類: {qp_instance.annotations['org.ommx.qplib.probtype']}")
print(f"目的関数の種類: {qp_instance.annotations['org.ommx.qplib.objtype']}")
print(f"目的関数の曲率: {qp_instance.annotations['org.ommx.qplib.objcurvature']}")
print(f"変数の数: {qp_instance.annotations['org.ommx.qplib.nvars']}")
print(f"制約の数: {qp_instance.annotations['org.ommx.qplib.ncons']}")